In [1]:
#!pip install h2o
!pip install auto-sklearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 1.9 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 3.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
import os
import gc
import glob
import random
import numpy as np 
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
from pathlib import Path
from itertools import groupby
import matplotlib.pyplot as plt

import joblib
import pickle

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

folder_path = "input"

In [25]:
de_train = pd.read_parquet(f'{folder_path}/de_train.parquet')
de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,-0.069476,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [26]:
id_map = pd.read_csv(f'{folder_path}/id_map.csv', index_col='id')
id_map

,cell_type,sm_name
id,,
0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,B cells,ABT-199 (GDC-0199)
2,B cells,ABT737
3,B cells,AMD-070 (hydrochloride)
4,B cells,AT 7867
...,...,...
250,Myeloid cells,Vandetanib
251,Myeloid cells,Vanoxerine
252,Myeloid cells,Vardenafil


In [27]:
genes = de_train.columns[5:]
genes

Index(['A1BG', 'A1BG-AS1', 'A2M', 'A2M-AS1', 'A2MP1', 'A4GALT', 'AAAS', 'AACS',
       'AAGAB', 'AAK1',
       ...
       'ZUP1', 'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', length=18211)

In [28]:
def add_columns(de_train, id_map):
    sm_lincs_id = de_train.set_index('sm_name')["sm_lincs_id"].to_dict()
    sm_name_to_smiles = de_train.set_index('sm_name')['SMILES'].to_dict()

    id_map['sm_lincs_id'] = id_map['sm_name'].map(sm_lincs_id)
    id_map['SMILES'] = id_map['sm_name'].map(sm_name_to_smiles)
    
    return id_map

add_columns(de_train, id_map)

,cell_type,sm_name,sm_lincs_id,SMILES
id,,,,
0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,LSM-47134,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C
1,B cells,ABT-199 (GDC-0199),LSM-45468,CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...
2,B cells,ABT737,LSM-1180,CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...
3,B cells,AMD-070 (hydrochloride),LSM-45591,NCCCCN(Cc1nc2ccccc2[nH]1)[C@H]1CCCc2cccnc21
4,B cells,AT 7867,LSM-1155,Clc1ccc(C2(c3ccc(-c4cn[nH]c4)cc3)CCNCC2)cc1
...,...,...,...,...
250,Myeloid cells,Vandetanib,LSM-1199,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1
251,Myeloid cells,Vanoxerine,LSM-2703,Fc1ccc(C(OCCN2CCN(CCCc3ccccc3)CC2)c2ccc(F)cc2)cc1
252,Myeloid cells,Vardenafil,LSM-2292,CCCc1nc(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CC...


In [29]:
def calculate_mae_and_mrrmse(y_true, y_pred, scaler=None):
    if scaler:
        y_true = scaler.inverse_transform(y_true)
        y_pred = scaler.inverse_transform(y_pred)

        # Calculate Mean Rowwise Root Mean Squared Error (MRRMSE)
        rowwise_rmse = np.sqrt(np.mean(np.square(y_true - y_pred), axis=1))
        mrrmse_score = np.mean(rowwise_rmse)
    else:
        # Calculate Mean Rowwise Root Mean Squared Error (MRRMSE)
        rowwise_rmse = np.sqrt(np.mean(np.square(y_true - y_pred), axis=1))
        mrrmse_score = np.mean(rowwise_rmse)
    
    # Print the results
    print(f"MRRMSE Score: {mrrmse_score}")

In [30]:
kf = KFold(n_splits=6, shuffle=True, random_state=6174)

# Create k-fold columns
for fold, (train_index, test_index) in enumerate(kf.split(de_train)):
    de_train.loc[test_index, 'kfold'] = fold

de_train['kfold'] = de_train['kfold'].astype(int)

# Reindex DataFrame to put 'kfold' as the first column
cols = ['kfold'] + [col for col in de_train if col != 'kfold']
de_train = de_train[cols]

de_train

,kfold,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,4,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,5,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,4,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,5,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,1,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,4,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,4,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,4,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,2,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [31]:
test_data = de_train[de_train["kfold"]==5].sample(frac=1, random_state=6174)
train_data = de_train[de_train["kfold"]!=5].sample(frac=1, random_state=6174)
train_data

,kfold,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
285,2,NK cells,GO-6976,LSM-1211,Cn1c2ccccc2c2c3c(c4c5ccccc5n(CCC#N)c4c21)CNC3=O,False,0.572913,-0.537372,-0.660346,-0.212915,...,0.693106,0.165283,-0.473416,0.004142,0.287328,0.270213,-0.789008,-0.075054,-0.010731,0.171419
511,2,T cells CD8+,Ruxolitinib,LSM-1139,N#CC[C@H](C1CCCC1)n1cc(-c2ncnc3[nH]ccc23)cn1,False,-0.391090,0.483317,-0.774944,0.967548,...,0.262297,0.191426,1.202725,0.988652,0.093701,0.600125,0.240327,0.994640,0.365673,-0.578786
111,3,T cells CD4+,Proscillaridin A;Proscillaridin-A,LSM-1476;LSM-5290,C[C@@H]1O[C@@H](O[C@@H]2C=C3CC[C@@H]4[C@H](CC[...,False,-0.767250,-0.163089,-0.646206,-3.473009,...,0.513831,0.581219,-0.434437,0.173986,1.400341,1.571638,-0.242477,-2.021973,-4.161092,0.953246
553,4,T cells CD4+,AT13387,LSM-45514,CC(C)c1cc(C(=O)N2Cc3ccc(CN4CCN(C)CC4)cc3C2)c(O...,False,0.277987,-0.201305,0.280088,-0.531355,...,0.058000,0.256395,0.854610,1.640502,2.172419,0.731250,-0.055159,-0.409620,-1.337875,0.113926
93,3,T regulatory cells,Vardenafil,LSM-2292,CCCc1nc(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CC...,False,-0.628011,0.676734,-0.412974,-0.754233,...,0.285048,-0.048031,-0.847719,0.488689,0.050510,-0.354516,-0.891566,-0.253083,0.643248,-0.318537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,3,T regulatory cells,TL_HRAS26,LSM-2013,Cc1onc(C(=O)N2CCN(C(c3ccc(Cl)cc3)c3ccc(Cl)cc3)...,False,-0.140583,0.463597,-0.148236,-0.031852,...,-1.051595,0.115485,-0.902081,-0.577548,0.105532,-0.323592,-0.186242,0.883351,0.145387,0.332967
504,3,T regulatory cells,BX 912,LSM-1055,O=C(Nc1cccc(Nc2ncc(Br)c(NCCc3c[nH]cn3)n2)c1)N1...,False,2.498582,0.970093,3.129195,3.045847,...,-0.253227,2.018453,0.979286,1.707055,2.291522,1.965766,1.423388,0.683576,-0.539229,0.566867
607,0,T cells CD4+,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,0.003888,0.548837,0.232179,-0.279816,...,-0.819877,0.757289,0.203455,0.053201,-0.571564,0.347606,0.060709,0.169867,-0.206451,0.365970
499,3,T cells CD8+,Sgc-cbp30,LSM-47437,COc1ccc(CCc2nc3cc(-c4c(C)noc4C)ccc3n2C[C@H](C)...,False,0.699837,-0.750765,0.014432,-1.243092,...,-0.125901,-0.069657,0.098344,0.007872,0.234029,-0.077685,-0.056956,-0.068496,-0.283606,-0.045120


In [35]:
import autosklearn.regression
from sklearn.model_selection import KFold
import numpy as np

# Define features and target variables
features = ['cell_type', 'sm_name']

# Prepare the data
X = train_data[features]
y = train_data[genes]


# Initialize the AutoML model with increased memory limit
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=360,
    per_run_time_limit=30,
    ensemble_kwargs={'ensemble_size': 1},
    memory_limit=1024*20,
    initial_configurations_via_metalearning=0
)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=6174)
scores = []

for train_index, val_index in kf.split(X):
		X_train, X_val = X.iloc[train_index], X.iloc[val_index]
		y_train, y_val = y.iloc[train_index], y.iloc[val_index]

		y_svd = TruncatedSVD(n_components=100, random_state=6174)
		z_y_train = y_svd.fit_transform(y_train)

		# Fit the AutoML model
		automl.fit(X_train, z_y_train)

		# Predict on validation set
		y_pred = automl.predict(X_val)
		y_pred = y_svd.inverse_transform(y_pred)

		# Evaluate predictions, replace 'calculate_mae_and_mrrmse' with your actual evaluation function
		score = calculate_mae_and_mrrmse(y_val.values, y_pred)
		scores.append(score)

# Print out the best model found by AutoML
print(automl.show_models())

# Calculate and print the mean of the cross-validation scores
print("Mean score across folds:", np.mean(scores))


/usr/local/lib/python3.9/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column cell_type has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column sm_name has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(


[WARNING] [2023-11-27 02:32:44,917:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-11-27 02:32:46,184:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-11-27 02:32:47,563:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-11-27 02:32:48,157:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-27 02:32:49,874:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
MRRMSE Score: 1.6879572084503505


ValueError: Unable to configure handler 'distributed_logfile'

In [21]:
#import h2o
#from h2o.automl import H2OAutoML
#import numpy as np
#from sklearn.model_selection import KFold

## Initialize H2O
#h2o.init()

## Define features and target variables
#features = ['cell_type', 'sm_name']

## Convert the data to H2OFrame
#hf = h2o.H2OFrame(train_data)

## Add a row index column to the H2OFrame
#hf['row_index'] = h2o.H2OFrame(np.arange(hf.nrow))

## Specify feature columns
#feature_cols = features

## Set up cross-validation using KFold
#kf = KFold(n_splits=5, shuffle=True, random_state=6174)

## Loop over each target gene
#for target in genes:
#    print(f"Training models for target: {target}")
#    scores = []
#    for train_index, val_index in kf.split(hf):
#        # Use H2O's filtering to create train and validation sets
#        train = hf[hf['row_index'].isin(h2o.H2OFrame(train_index).as_data_frame().values.flatten())]
#        valid = hf[hf['row_index'].isin(h2o.H2OFrame(val_index).as_data_frame().values.flatten())]

#        # Create and train H2OAutoML model for the current target
#        automl = H2OAutoML(max_runtime_secs=360, seed=1, project_name=f"automl_{target}")
#        automl.train(x=feature_cols, y=target, training_frame=train, validation_frame=valid)

#        # Get the leader model and predict on validation data
#        leader_model = automl.leader
#        pred = leader_model.predict(valid[feature_cols])

#        # Calculate and store the score for this fold
#        # Replace 'calculate_mae_and_mrrmse' with your actual evaluation function
#        score = calculate_mae_and_mrrmse(valid[target].as_data_frame().values, pred.as_data_frame().values)
#        scores.append(score)

#    # Calculate and print the average score across all folds for the current target
#    mean_score = np.mean(scores)
#    print(f"Mean Score for {target}: {mean_score}")

## Shutdown H2O - this will remove all H2O data and models
#h2o.shutdown()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6nu0_4l2
  JVM stdout: /tmp/tmp6nu0_4l2/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6nu0_4l2/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_ryazwh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,9.98 Gb
H2O_cluster_total_cores:,6
H2O_cluster_allowed_cores:,6
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Training models for target: A1BG
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


H2OTypeError: Argument `rhs` should be a string | numeric | date | pandas.Timestamp | numpy.datetime64 | H2OFrame, got ndarray [  0   1   2   3   5   8   9  10  11  13  15  16  17  18  19  20  21  22
  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
  44  45  46  47  48  49  51  52  53  54  55  56  58  61  62  63  64  65
  69  70  71  72  73  74  76  79  80  81  82  83  84  85  87  88  89  90
  91  92  93  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 121 122 124 125 127 129 130 131 132 137
 139 140 142 143 144 145 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167 168 169 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 197
 201 203 205 207 208 209 210 211 212 213 214 215 216 217 218 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 237 238 240 241 242 243 245
 246 247 248 249 251 252 253 254 257 258 259 260 261 262 264 265 266 2

Closing connection _sid_8d8c at exit
H2O session _sid_8d8c closed.


In [12]:
h2o.shutdown()

H2O session _sid_a92e closed.


/tmp/ipykernel_1248/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


# Submission

In [12]:
# Training with whole data.

features = ['cell_type', 'sm_name']

models = []
svds = []

for i in range(5):
    train_index = de_train[de_train["kfold"]!=i].index
    val_index = de_train[de_train["kfold"]==i].index

    train_df = de_train.loc[train_index].sample(frac=1, random_state=6174)
    val_df = de_train.loc[val_index].sample(frac=1, random_state=6174)

    # genes
    pred_df = val_df.copy()
    pred_df[genes] = 0

    # Model
    model = make_pipeline(
        ColumnTransformer([('ohe', OneHotEncoder(handle_unknown='ignore'), ['sm_name'])]),
        ElasticNet(alpha = 0.5)
    )
    svd = TruncatedSVD(n_components=n_components, random_state=6174)
    z_tr = svd.fit_transform(train_df[genes])
    model.fit(train_df[features], z_tr)
    models.append(model)
    svds.append(svd)
    
    y_pred = svd.inverse_transform(model.predict(val_df[features]))
    pred_df[genes] = y_pred

    # Get validation score
    calculate_mae_and_mrrmse(val_df[genes].values, pred_df[genes].values)

MRRMSE Score: 1.2062788655714691
MRRMSE Score: 1.3004105117887168
MRRMSE Score: 1.5331703092543012
MRRMSE Score: 1.201409506012575
MRRMSE Score: 1.4069196204491876


In [13]:
y_preds = []
for i in range(5):
    y_preds.append(svds[i].inverse_transform(models[i].predict(id_map[features])))

y_preds = np.array(y_preds)
id_map.loc[:, genes] = y_preds.mean(axis=0)
id_map = id_map.loc[:, genes]
id_map.to_csv('submission.csv')
id_map

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.366438,0.240036,0.147519,0.161995,0.684819,0.934407,0.002626,0.338118,-0.003565,0.065788,...,0.064906,0.148070,0.027723,0.245517,0.480480,0.380667,0.238002,0.155720,-0.164395,-0.065039
1,0.365069,0.239189,0.146672,0.161155,0.682371,0.930887,0.002360,0.337212,-0.003767,0.065919,...,0.064698,0.147403,0.027184,0.244436,0.479101,0.379552,0.237545,0.155506,-0.164340,-0.065097
2,0.366438,0.240036,0.147519,0.161995,0.684819,0.934407,0.002626,0.338118,-0.003565,0.065788,...,0.064906,0.148070,0.027723,0.245517,0.480480,0.380667,0.238002,0.155720,-0.164395,-0.065039
3,0.362930,0.237853,0.145549,0.160017,0.678539,0.925228,0.001966,0.335847,-0.004091,0.066059,...,0.064388,0.146377,0.026406,0.242750,0.476999,0.377850,0.236888,0.155191,-0.164288,-0.065169
4,0.363709,0.238368,0.145880,0.160380,0.679947,0.927337,0.002106,0.336353,-0.003971,0.065999,...,0.064512,0.146750,0.026671,0.243361,0.477764,0.378466,0.237112,0.155303,-0.164311,-0.065150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.358078,0.234847,0.142920,0.157394,0.669875,0.912324,0.001060,0.332732,-0.004810,0.066437,...,0.063674,0.144095,0.024596,0.238934,0.472214,0.373996,0.235380,0.154462,-0.164098,-0.065337
251,0.364442,0.238822,0.146316,0.160814,0.681259,0.929237,0.002247,0.336832,-0.003861,0.065937,...,0.064620,0.147106,0.026953,0.243940,0.478497,0.379060,0.237351,0.155416,-0.164336,-0.065121
252,0.361440,0.236931,0.144817,0.159292,0.675893,0.921140,0.001694,0.334908,-0.004307,0.066172,...,0.064170,0.145711,0.025864,0.241589,0.475554,0.376694,0.236450,0.154969,-0.164210,-0.065212
